In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
import torch
import torch.nn

In [2]:
attri_data = pd.read_csv('../LiterallyWikidata/files_needed/numeric_literals_ver06')

In [3]:
x = attri_data[['e','a']].to_numpy()
y = attri_data['std_v'].to_numpy()

In [65]:
num_lit = np.load('../LiterallyWikidata/files_needed/num_lit.npy')

In [79]:
with open('../LiterallyWikidata/files_needed/attribute.txt','r') as fr:
    att2idx = {line.strip(): i for i, line in enumerate(fr)}
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
    ent2idx ={line.strip(): i for i, line in enumerate(f)}

In [82]:
num_lit[0][0]
## constraint needed:
pop_idx = att2idx['P1082']
gdp = att2idx['P4010']
nominal_gdp = att2idx['P2131']
nominal_gdp_per = att2idx['P2132']
gdp_per = att2idx['P2299']
date_of_birth = att2idx['P569']
date_of_death = att2idx['P570']
area = ['P2046']
net_profit = att2idx['P2295']
retirement_age = att2idx['P3001']
age_of_majority = att2idx['P2997']
work_start = att2idx['P2031']
work_end = att2idx['P2032']
gdp

25

In [6]:
emb_ent = torch.load('../LiterallyWikidata/files_needed/pretrained_kge/pretrained_complex_entemb.pt')
print(emb_ent.shape)
list_ent_ids =[]
with open('../LiterallyWikidata/files_needed/list_ent_ids.txt','r') as f:
    for line in f:
        list_ent_ids.append(line.strip())
print(len(list_ent_ids))


#attri_data['e'][0] --> find list idx --> emb.ent[idx]
ent2idx = {e:i for i,e in enumerate(list_ent_ids)}
attri_data['list_idx']=attri_data['e'].map(ent2idx)


weight = emb_ent
embedding = torch.nn.Embedding.from_pretrained(weight)
input = torch.LongTensor(attri_data['list_idx'].to_numpy())


torch.Size([47998, 128])
47998


In [5]:
attri_data[:5]

,e,a,v,name_e,name_a,ent_type,std_v,list_idx
0,Q1000056,P1082,11032.0,Sušice,population,Q7841907,-0.030708,5
1,Q1000056,P2044,472.0,Sušice,elevation above sea level,Q7841907,-0.031042,5
2,Q1000056,P2046,45630000.0,Sušice,area,Q7841907,1.411756,5
3,Q1000138,P1082,1375.0,Cantenac,population,Q484170,-0.031014,8
4,Q1000138,P2044,1.0,Cantenac,elevation above sea level,Q484170,-0.031057,8


In [8]:
emb_e = embedding(input)

In [18]:
len(attri_data['a'].unique())

81

In [4]:
#attri_data['e'][0] --> find list idx --> emb.ent[idx]
att2idx = {a:i for i,a in enumerate(attri_data['a'].unique())}
attri_data['a_idx']=attri_data['a'].map(att2idx)


embedding_a = torch.nn.Embedding(len(attri_data['a'].unique()),128,padding_idx=0)
input_a = torch.LongTensor(attri_data['a_idx'].to_numpy())
emb_a = embedding_a(input_a)
# ent embedding use not pretrain
ent2idx = {e:i for i,e in enumerate(attri_data['e'].unique())}
attri_data['e_idx']=attri_data['e'].map(ent2idx)


embedding_e = torch.nn.Embedding(len(attri_data['e'].unique()),128,padding_idx=0)
input_e = torch.LongTensor(attri_data['e_idx'].to_numpy())
emb_e = embedding_e(input_e)

In [5]:
x = torch.cat([emb_e,emb_a],dim=1).detach().numpy()

In [6]:
y= attri_data.loc[:,'std_v']

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=802,stratify=x[:,1])

In [38]:
X_train[:,1]

array(['P1332_Longtiude', 'P625_Longtiude', 'P569', ..., 'P570', 'P582',
       'P1619'], dtype=object)

In [8]:
nn=MLPRegressor(hidden_layer_sizes=(100,100,64),  #預設值1層,數目100
         activation='tanh',  #{'identity'(回歸)無，'logistic'為sigmoid，'tanh'，'relu'}，預設='relu'
         solver='sgd',   #{'lbfgs'牛頓，'sgd'隨機梯度，'adam'隨機梯度優化}，預設='adam'
         batch_size=200,  #'auto'為min(200～樣本數)
         learning_rate='constant',  #constant固定'，'invscaling隨著時間遞減'，'adaptive誤差減少時不改變'}，預設 ='constant' 
         learning_rate_init=0.001, #優化器為sgd、adam使用
         power_t=0.5,     # 優化器為sgd，則可以設定強化學習率   
         max_iter=35,    #訓練次數
         shuffle=True,    #隨機設定權重
         random_state=1,   #隨機的基礎值
         momentum=0.9)

In [9]:
nn.fit(X_train, y_train)
y_predtrain=nn.predict(X_train)

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [10]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_train, y_predtrain)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_train, y_predtrain)
print('MAE=',MAE)

R2=1-MSE/np.var(y_predtrain)
print("R2:", R2)

MSE= 1.6690571357168333e+40
RMSE= 1.2919199416824686e+20
MAE= 3.3812661755217787e+18
R2: 1.0


/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/numpy/core/_methods.py:244: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [13]:
ytrain = []
for value in y_train:
    ytrain.append(value)
count=0
errvalue=abs((ytrain-y_predtrain)/ytrain)
errvalue

array([[ 0.14053851, -0.01507521,  0.41197178, ...,  0.6470181 ,
        -0.52527577,  0.1809235 ],
       [-2.0111618 , -0.2368167 , -1.2115607 , ..., -0.42620778,
        -0.678838  , -0.41619265],
       [-0.54481333, -1.2218765 ,  0.7915829 , ..., -0.2657716 ,
        -0.5661416 , -0.47609696],
       ...,
       [-1.3330302 ,  1.01035   , -0.18121403, ..., -0.2657716 ,
        -0.5661416 , -0.47609696],
       [ 0.2975672 , -0.82920057, -0.39734676, ..., -0.9954493 ,
        -0.4204575 , -0.48098677],
       [ 1.014133  , -0.71977836, -0.8285394 , ...,  0.6470181 ,
        -0.52527577,  0.1809235 ]], dtype=float32)

In [14]:
threshold=0.02  #門檻值
for i in range(len(errvalue)):
  if errvalue[i]<threshold:           
    count=count+1
print("通過門檻筆數",count)
print("通過門檻百分比",count/len(errvalue))

通過門檻筆數 0
通過門檻百分比 0.0


In [15]:
y_pred=nn.predict(X_test)
y_pred

array([ 3.3936745e+18, -8.4060473e+18, -4.6824379e+18, ...,
        6.3630766e+18, -8.6592055e+18, -4.6873693e+18], dtype=float32)

In [17]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_test, y_pred)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_test, y_pred)
print('MAE=',MAE)

R2=1-MSE/np.var(y_test)
print("R2:", R2)

ytest = []
for value in y_test:
    ytest.append(value)
count=0
errvalue=abs((ytest-y_pred)/ytest)
ytest

MSE= 1.6606402846747535e+37
RMSE= 4.075095440201068e+18
MAE= 3.131321383328423e+18
R2: -2359429440.4139247


[0.4097233,
 -0.031057097,
 -0.031055795,
 0.1035596,
 -0.031056497,
 -0.013318526,
 1.7927614,
 -0.03103063,
 0.2202663,
 -0.031057097,
 -0.030995345,
 0.41920927,
 -0.031056767,
 -0.031055877,
 -0.02919045,
 -0.031053385,
 -0.030997146,
 -0.019384412,
 -0.03105517,
 -0.030993668,
 -0.031055989,
 -0.02879805,
 -0.031037271,
 -0.03105725,
 -0.031055918,
 -0.030993763,
 -0.03105692,
 -0.031057192,
 -0.031059837,
 -0.031041477,
 -0.031055937,
 -0.0310552,
 -0.015247336,
 -0.030971756,
 -0.031055806,
 -0.031048339,
 -0.031056898,
 -0.030903583,
 -0.030995881,
 -0.031056914,
 -0.031057235,
 0.6645928,
 -0.031040719,
 -0.031028418,
 -0.031056656,
 -0.030961446,
 -0.0310572,
 -0.031056145,
 -0.031056304,
 -0.031006157,
 -0.030994078,
 -0.030994965,
 0.30189964,
 -0.031055043,
 -0.03105697,
 -0.03105691,
 -0.03072844,
 -0.031056855,
 -0.031056806,
 4.655003,
 -0.031052798,
 -0.031056836,
 274.9297,
 -0.031056968,
 -0.031029461,
 -0.031055896,
 -0.031032085,
 0.25088724,
 -0.030994173,
 -0.031

In [ ]:
threshold=0.01  #門檻值
for i in range(len(errvalue)):
  if errvalue[i]<0.01:           
    count=count+1
print("通過門檻筆數",count)
print("通過門檻百分比",count/len(errvalue))

In [41]:
from sklearn import linear_model
reg = linear_model.Ridge(alpha=.52)#調整alpha值決定正則化的強度
reg.fit(X_train, y_train)
#print(reg.coef_) # 印出訓練後的模型參數，數值都明顯小於單純的線性回歸
y_predtrain = reg.predict(X_train)

[-4.41818510e+17 -1.06328932e+18  2.38763898e+17 -2.27810288e+17
 -6.71998228e+16  1.59245327e+17  2.43395539e+17 -1.34701266e+18
  1.88870723e+17 -2.24269362e+17 -3.85441704e+17 -1.03717550e+17
 -1.86330868e+17 -1.09377536e+17 -1.68096585e+17 -2.68021614e+16
  8.93950026e+16 -2.11440195e+18 -7.60218660e+16 -9.97186351e+16
 -2.34964776e+17  1.47258520e+17  4.13949601e+17 -1.30029156e+17
  2.99178798e+17 -8.94582331e+16 -4.28834343e+17  6.29185427e+17
  3.65651560e+17 -2.88418393e+17  8.19814553e+15  1.23832222e+18
  3.63595336e+17 -4.77494260e+17  5.45453205e+18  3.00302705e+17
  3.64907981e+17 -4.70638496e+17  2.18309425e+17 -5.87628320e+16
  2.58692082e+17 -1.91792830e+17 -1.16482328e+19 -4.01681800e+17
 -5.50610719e+17 -4.27734522e+17 -2.97533344e+17 -2.21440577e+17
 -2.61584790e+16  7.27391882e+16 -4.37956054e+16  1.05499941e+18
  8.44906654e+16  7.23540636e+17  1.84905987e+17 -7.23441199e+17
  4.57605607e+17 -7.74728262e+17 -1.05301835e+17 -3.50424354e+17
  6.73793439e+16  3.92967

/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.75972e-08): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


In [42]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import numpy as np

MSE = mean_squared_error(y_train, y_predtrain)
print('MSE=',MSE)
RMSE =np.sqrt(MSE)
print('RMSE=',RMSE)
MAE= mean_absolute_error(y_train, y_predtrain)
print('MAE=',MAE)

R2=1-MSE/np.var(y_predtrain)
print("R2:", R2)

MSE= 3.220578710718283e+40
RMSE= 1.7945970886854472e+20
MAE= 6.87268669111927e+18
R2: 1.0


/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/numpy/core/_methods.py:233: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/projekte/tcl/tclext/kgc_chu/neuenv/lib/python3.7/site-packages/numpy/core/_methods.py:244: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [44]:
from sklearn.linear_model import RidgeCV
rcv = RidgeCV(alphas=np.array([.3, .4, .5, .6]))
rcv.fit(X_train, y_train)
#RidgeCV(alphas=array([0.1, 0.2, 0.3, 0.4]), cv=None, fit_intercept=True,
       # gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)
print(rcv.alpha_)

0.6


In [52]:
rcv2 = RidgeCV(alphas=np.array([.63, .64, .65, .66, .67]))
rcv2.fit(X_train, y_train)
print(rcv2.alpha_)

0.67


In [46]:
alphas_to_test = np.linspace(0.01, 1)
rcv3 = RidgeCV(alphas=alphas_to_test, store_cv_values=True)
rcv3.fit(X_train, y_train)

RidgeCV(alphas=array([0.01      , 0.03020408, 0.05040816, 0.07061224, 0.09081633,
       0.11102041, 0.13122449, 0.15142857, 0.17163265, 0.19183673,
       0.21204082, 0.2322449 , 0.25244898, 0.27265306, 0.29285714,
       0.31306122, 0.33326531, 0.35346939, 0.37367347, 0.39387755,
       0.41408163, 0.43428571, 0.4544898 , 0.47469388, 0.49489796,
       0.51510204, 0.53530612, 0.5555102 , 0.57571429, 0.59591837,
       0.61612245, 0.63632653, 0.65653061, 0.67673469, 0.69693878,
       0.71714286, 0.73734694, 0.75755102, 0.7777551 , 0.79795918,
       0.81816327, 0.83836735, 0.85857143, 0.87877551, 0.89897959,
       0.91918367, 0.93938776, 0.95959184, 0.97979592, 1.        ]),
        store_cv_values=True)

In [47]:
rcv3.cv_values_.shape

(122474, 50)

In [53]:
def MAD(target, predictions):
    absolute_deviation = np.abs(target - predictions)
    return absolute_deviation.mean()

In [54]:
import sklearn
MAD = sklearn.metrics.make_scorer(MAD, greater_is_better=False)
rcv4 = RidgeCV(alphas=alphas_to_test, store_cv_values=True,scoring=MAD)
rcv4.fit(X_train, y_train)
smallest_idx = rcv4.cv_values_.mean(axis=0).argmin()
alphas_to_test[smallest_idx]

1.0